# VANGUARD AB TEST


## METADATA HELP

This comprehensive set of fields will guide your analysis, helping you unravel the intricacies of client behavior and preferences.

- **client_id**: Every client’s unique ID.
- **variation**: Indicates if a client was part of the experiment.
- **visitor_id**: A unique ID for each client-device combination.
- **visit_id**: A unique ID for each web visit/session.
- **process_step**: Marks each step in the digital process.
- **date_time**: Timestamp of each web activity.
- **clnt_tenure_yr**: Represents how long the client has been with Vanguard, measured in years.
- **clnt_tenure_mnth**: Further breaks down the client’s tenure with Vanguard in months.
- **clnt_age**: Indicates the age of the client.
- **gendr**: Specifies the client’s gender.
- **num_accts**: Denotes the number of accounts the client holds with Vanguard.
- **bal**: Gives the total balance spread across all accounts for a particular client.
- **calls_6_mnth**: Records the number of times the client reached out over a call in the past six months.
- **logons_6_mnth**: Reflects the frequency with which the client logged onto Vanguard’s platform over the last six months.


In [2]:
%load_ext autoreload
%autoreload 2 

In [3]:
from cleaning import *
from mining import *
from db_handling import *
import pandas as pd
from dotenv import load_dotenv
import os


In [ ]:
# Load environment variables
load_dotenv()

### Load Configuration

In [ ]:
# Load config.yaml
config = parse_config()

## Data Mining

In [6]:
# Creates a dictionary of all imported dataframes
dataframes = { name:import_data_from_config(config, name) for name in config['tables']}

## Data Cleaning

In [7]:
#TODO: don't impose categories?

In [8]:
# Rename columns
dataframes = rename_columns(dataframes, config)

In [9]:
# Select columns
dataframes = select_columns(dataframes, config)

In [ ]:
display_dataFrames(dataframes,'head')

## Separation

In [ ]:
client_df = dataframes['clients']
experiment_df = dataframes['experiment']
visits_df = dataframes['visits']
display(experiment_df['variation'].isna().sum())
display (client_df, experiment_df, visits_df)

In [ ]:
variation_visits = visits_df.merge(experiment_df, on='client_id')
display(variation_visits['variation'].value_counts())
display(variation_visits)
confirmed_steps = variation_visits[variation_visits['process_step'] == 'confirm']
display(confirmed_steps)
unique_visit_ids = confirmed_steps.drop_duplicates(subset='visit_id')
display(unique_visit_ids)
""" visits = variation_visits.groupby(['variation','process_step']).agg({'process_step':'count'})
visits """


In [ ]:

display(unique_visit_ids['variation'].value_counts())

In [ ]:
# drop the nulls from clients, but keep the list of the drops

nulls_client_id = client_df[client_df.isna().any(axis=1)]['client_id']
nulls_client_id

In [ ]:
client_df = client_df.dropna(axis=0)
client_df

In [ ]:
display(client_df['gender'].value_counts(dropna = False))
# x->u, keep 'U's for everything except the gender statistics

In [17]:
#client_df['gender'] = client_df['gender'].replace(to_replace=r'.*X.*', value ="U", regex=True)

In [ ]:
display(experiment_df['variation'].value_counts(dropna = False))
# keep NaN for general analysis of clients, but drop them from everywhere for test analysis

In [ ]:
# client_df, experiment_df, visit_df -> for general analysis
# new_client_df, new_experiment_df, new_visit_dfn -> for test/control analysis   experiment_df_null = 
nulls_in_experiment = experiment_df[experiment_df.isna().any(axis=1)]['client_id']
nulls_in_experiment

In [ ]:
# new df removing client ID that are null in experiment

new_experiment_df = experiment_df[~experiment_df['client_id'].isin(nulls_in_experiment)]
display(new_experiment_df.count())

new_experiment_df = new_experiment_df[~new_experiment_df['client_id'].isin(nulls_client_id)]
display(new_experiment_df.count())



In [ ]:
new_visits_df = visits_df[~visits_df['client_id'].isin(nulls_in_experiment)]
new_visits_df

In [ ]:
new_client_df = client_df[~client_df['client_id'].isin(nulls_in_experiment)]
new_client_df

## End separation

In [23]:
dataframes['clients'] = new_client_df.copy()
dataframes['experiment'] = new_experiment_df.copy()
dataframes['visits'] = new_visits_df.copy()

In [24]:
# Data Categorizing
dataframes = clean_categorical_data(dataframes, config)

In [25]:
#Convert types
dataframes = convert_types(dataframes, config)

In [ ]:
display_dataFrames(dataframes, 'head', 'dtypes', 'cat_count')

In [27]:
client_df = dataframes['clients']
experiment_df = dataframes['experiment']
visits_df = dataframes['visits']


### SQL EXPORT

In [28]:
if config['refresh_db']:

    db_password = os.getenv('SQL_PASSWORD')

    # Create database if it doesn't exist
    engine = create_db(db_password, config)

    # Export tables to database if refresh is set to true
    export_dataframes_to_sql(engine, dataframes)

    # Import data from database
    dataframes = import_all_tables_from_sql(engine)

### Local Caching

In [ ]:
""" # Save files locally in an untracked folder
export_dataframes_to_csv(dataframes) """

In [ ]:
#TODO CAREFUL DATA WONT BE PROPERLY CATEGORIZED / TYPED run after : convert_types(dataframes, config)
""" clients_df = pd.read_csv('data/cleaned/clients.csv')
experiment_df = pd.read_csv('data/cleaned/experiment.csv')
visits_df = pd.read_csv('data/cleaned/visits.csv') """

## CLEAN FRAMES

In [39]:
display('clients :',client_df, 'experiment :',experiment_df, 'visits :',visits_df)

experiment_df['variation'].value_counts()


'clients :'

,client_id,client_since_month,client_age,gender,number_of_accounts,balance,calls_6_months,logons_6_month
0,836976,73,60,U,2,45105.30,6,9
1,2304905,94,58,U,2,110860.30,6,9
2,1439522,64,32,U,2,52467.79,6,9
3,1562045,198,49,M,2,67454.65,3,6
4,5126305,145,33,F,2,103671.75,0,3
...,...,...,...,...,...,...,...,...
50495,1780858,262,68,M,3,372100.59,6,9
50496,6967120,260,68,M,3,4279873.38,6,9
50497,5826160,249,56,F,2,44837.16,2,5
50498,8739285,229,69,F,2,44994.24,1,4


'experiment :'

,client_id,variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
...,...,...
50495,393005,Control
50496,2908510,Control
50497,7230446,Test
50498,5230357,Test


'visits :'

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
755400,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
755401,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
755402,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
755403,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


variation
Test       26961
Control    23526
unknown        0
Name: count, dtype: int64

In [32]:
# client_since_year : redundant : drop
# client_since_month: hypothesis : the longer they are client, the more valuable to us
# client_since_month: hypothesis : the older the client is, the more valuable to us
# gender: hypothesis : the men have more balance
# number_of_accounts: hypothesis : the clients with more accounts have more balance
# calls + logons : hypothesis : active clients are more valuable to us

# process steps + time : 
    # - SUCCESS : All the steps, in order, in a reasonable amount of time for each step
    
    # - ERROR : path do not start with start : drop
    # - ERROR : path do not complete : analyse
    # - ERROR : path do not complete in order: analyse
    # - ERROR : All the steps in order but took very long
    # - ERROR : Unusual amount of time between steps

## Data Exploration

In [43]:
visits_variations = visits_df.merge(experiment_df, on='client_id', how='right')
display(visits_variations)

,client_id,visitor_id,visit_id,process_step,date_time,variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test
...,...,...,...,...,...,...
321190,6334360,629124187_65258232847,586047816_14599436811_389185,step_2,2017-04-11 08:45:38,Test
321191,6334360,629124187_65258232847,586047816_14599436811_389185,step_1,2017-04-11 08:44:43,Test
321192,6334360,629124187_65258232847,586047816_14599436811_389185,step_2,2017-04-11 08:44:30,Test
321193,6334360,629124187_65258232847,586047816_14599436811_389185,step_1,2017-04-11 08:42:36,Test


variation
Test       177787
Control    143408
unknown         0
Name: count, dtype: int64

In [34]:
# Handle outliers

In [35]:
#frequency tables

## Analysis

In [36]:
#TODO: consider binning / pd.cut / qcut for numerical data
#TODO: correlation matrix
#TODO: tukeys_test_outliers

In [37]:
# check back and forth between steps, lost?
# 

## Visualizations

## Conclusions